# Push Docker Image to AWS Elastic Container Registry

In [ ]:
# for full build and push for use with containerized sagemaker training jobs
!cd container; ./build_and_push.sh drug-prediction-gnn-gpu

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
Sending build context to Docker daemon  153.1kB
Step 1/20 : ARG REGION=us-west-2
Step 2/20 : FROM nvidia/cuda:10.1-base-ubuntu16.04
 ---> 6075bdccdb5b
Step 3/20 : RUN apt-get update
 ---> Using cache
 ---> d08bc5b77a37
Step 4/20 : RUN apt-get -y install curl
 ---> Using cache
 ---> 880551567752
Step 5/20 : RUN curl -LO http://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
 ---> Using cache
 ---> 01e606c9df8e
Step 6/20 : RUN bash Miniconda3-latest-Linux-x86_64.sh -p /miniconda -b
 ---> Using cache
 ---> 23ba6b36d626
Step 7/20 : RUN rm Miniconda3-latest-Linux-x86_64.sh
 ---> Using cache
 ---> a0263234dde2
Step 8/20 : ENV PATH=/miniconda/bin:${PATH}
 ---> Using cac

In [72]:
# for local building
!cd container; docker build -t drug-prediction-gnn-gpu .

Sending build context to Docker daemon  153.1kB
Step 1/20 : ARG REGION=us-west-2
Step 2/20 : FROM nvidia/cuda:10.1-base-ubuntu16.04
 ---> 6075bdccdb5b
Step 3/20 : RUN apt-get update
 ---> Using cache
 ---> d08bc5b77a37
Step 4/20 : RUN apt-get -y install curl
 ---> Using cache
 ---> 880551567752
Step 5/20 : RUN curl -LO http://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
 ---> Using cache
 ---> 01e606c9df8e
Step 6/20 : RUN bash Miniconda3-latest-Linux-x86_64.sh -p /miniconda -b
 ---> Using cache
 ---> 23ba6b36d626
Step 7/20 : RUN rm Miniconda3-latest-Linux-x86_64.sh
 ---> Using cache
 ---> a0263234dde2
Step 8/20 : ENV PATH=/miniconda/bin:${PATH}
 ---> Using cache
 ---> c6d55a188946
Step 9/20 : RUN conda update -y conda
 ---> Using cache
 ---> fbe6838a3003
Step 10/20 : RUN apt-get update
 ---> Using cache
 ---> 8ce26ef9856a
Step 11/20 : RUN apt-get install -y build-essential
 ---> Using cache
 ---> bffffa7098cc
Step 12/20 : RUN conda install -y -c conda-forge rdkit==2018

In [2]:
import torch

torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

device(type='cuda')

# Run Local Training Job

In [ ]:
# Download the dataset to local

# download pretrained mol2vec model
!wget -O data/model_300dim.pkl https://github.com/samoturk/mol2vec/raw/master/examples/models/model_300dim.pkl model_300dim.pkl   

# download example data     
!wget -O data/ames.sdf https://github.com/samoturk/mol2vec/raw/master/examples/data/ames.sdf
    
#download hiv data
!wget -O data/hiv.zip https://s3-us-west-1.amazonaws.com/deepchem.io/datasets/molnet_publish/hiv.zip
!cd data; unzip hiv.zip

In [69]:
from sagemaker import get_execution_role

role = get_execution_role()

In [73]:
import os
import subprocess

instance_type = 'local'

if subprocess.call('nvidia-smi') == 0:
    ## Set type to GPU if one is present
    instance_type = 'local_gpu'
    
print("Instance type = " + instance_type)

Instance type = local_gpu


In [74]:
from sagemaker.estimator import Estimator

# hyperparameters = {'dev-mode':True, 
#                    'net-type': 'GAT',
#                    'epochs': 200,
#                    'learning-rate':1e-4,
#                    'gcn-hidden-feats':64,
#                    'classifier-hidden-feats':64,
#                    'batch-size':200,
#                    'num-hidden-layers':2,
#                    'random-state':-1}

# hyperparameters = {'dev-mode':True, 
#                    'net-type': 'GCN',
#                    'epochs': 200,
#                    'learning-rate':1e-4,
#                    'gcn-hidden-feats':128,
#                    'classifier-hidden-feats':512,
#                    'batch-size':200,
#                    'num-hidden-layers':2}

hyperparameters = {'dev-mode':True, 
                   'net-type': 'GCN',
                   'epochs': 2,
                   'learning-rate':1e-4,
                   'batch-size':200,
                   'num-hidden-layers':20,
                   'random-state':42}

estimator = Estimator(role=role,
                      train_instance_count=1,
                      train_instance_type=instance_type,
                      image_name='drug-prediction-gnn-gpu:latest',
                      hyperparameters=hyperparameters)

estimator.fit('file://data')

Creating tmpw9wj71ur_algo-1-7zm6g_1 ... 
Attaching to tmpw9wj71ur_algo-1-7zm6g_12mdone
algo-1-7zm6g_1  | 2020-02-07 01:54:00,618 sagemaker-containers INFO     Failed to parse hyperparameter dev-mode value True to Json.
algo-1-7zm6g_1  | Returning the value itself
algo-1-7zm6g_1  | 2020-02-07 01:54:00,618 sagemaker-containers INFO     Failed to parse hyperparameter net-type value GCN to Json.
algo-1-7zm6g_1  | Returning the value itself
algo-1-7zm6g_1  | 2020-02-07 01:54:00,634 sagemaker-containers INFO     Failed to parse hyperparameter dev-mode value True to Json.
algo-1-7zm6g_1  | Returning the value itself
algo-1-7zm6g_1  | 2020-02-07 01:54:00,634 sagemaker-containers INFO     Failed to parse hyperparameter net-type value GCN to Json.
algo-1-7zm6g_1  | Returning the value itself
algo-1-7zm6g_1  | 2020-02-07 01:54:00,657 sagemaker-containers INFO     Failed to parse hyperparameter dev-mode value True to Json.
algo-1-7zm6g_1  | Returning the value itself
algo-1-7zm6g_1  | 2020-02-07 0

# Run A Sagemaker Training Job

In [ ]:
# build and push image for sagemaker training jobs
!cd container; ./build_and_push.sh drug-prediction-gnn-gpu

In [ ]:
import sagemaker as sage

sess = sage.Session()

In [ ]:
import boto3

client = boto3.client('sts')
account = client.get_caller_identity()['Account']

my_session = boto3.session.Session()
region = my_session.region_name

algorithm_name = 'drug-prediction-gnn-gpu'

ecr_image = '{}.dkr.ecr.{}.amazonaws.com/{}:latest'.format(account, region, algorithm_name)

print(ecr_image)

In [ ]:
from sagemaker.estimator import Estimator

data_location = 's3://jdurago-insight-2020a/data'
output_path = 's3://jdurago-insight-2020a/output/baseline/'
max_run_time = 3*60*60 # train for max of 3 hours

hyperparameters = {'dev-mode':False, 
                   'net-type': 'GCN',
                   'epochs': 200,
                   'learning-rate':1e-4,
                   'gcn-hidden-feats':128,
                   'classifier-hidden-feats':512,
                   'batch-size':200,
                   'num-hidden-layers':6}

# experiment for comparing layers
hyperparameters = {'dev-mode':False, 
                   'net-type': 'GCN',
                   'epochs': 2,
                   'learning-rate':1e-4,
                   'batch-size':200,
                   'num-hidden-layers':6,
                   'random-state':42}


# defaults
# hyperparameters = {'dev-mode':False, 
#                    'net-type': 'GCN',
#                    'epochs': 200,
#                    'learning-rate':1e-4,
#                    'gcn-hidden-feats':64,
#                    'gat-hidden-feats':32,
#                    'num-hidden-layers':2,
#                    'num-head-layers':2,
#                    'head-output-size':4,
#                    'classifier-hidden-feats':64,
#                    'batch-size':200,
#                    'num-hidden-layers':6}

base_job_name = hyperparameters['net-type'] + '-layer-exp-' + str(hyperparameters['num-hidden-layers'])

metric_definitions = [{"Name": "train:loss", 
                       "Regex": "train_loss:(\d+.\d+)"}, 
                      {"Name": "test:auc_roc", 
                       "Regex": "test_score:(\d+.\d+)"},
                      {"Name": "val:auc_roc", 
                       "Regex": "auc_roc:(\d+.\d+)"},
                      {"Name": "val:f1_score", 
                       "Regex": "f1_score:(\d+.\d+)"}
                     ]

instance_type = 'ml.m5.12xlarge'

tags = [{'Key': 'net-type',
         'Value': hyperparameters['net-type']}]

estimator = Estimator(role=role,
                      base_job_name = base_job_name,
                      train_instance_count=1,
                       train_instance_type=instance_type,
                      image_name=ecr_image,
                      hyperparameters=hyperparameters,
                     output_path = output_path,
                     train_max_run=max_run_time,
                     metric_definitions=metric_definitions,
                     tags=tags)

estimator.fit(data_location)


# Hyperparameter Tuning

In [ ]:
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner
from sagemaker.estimator import Estimator
import sagemaker as sage

sess = sage.Session()

max_jobs = 5
max_parallel_jobs = 4

data_location = 's3://jdurago-insight-2020a/data'
output_path = 's3://jdurago-insight-2020a/output/baseline/'
max_run_time = 6*60*60 # train for max of 6 hours
instance_type = 'ml.m5.4xlarge'


hyperparameters = {'dev-mode':False, 
                   'net-type': 'GCN',
                   'epochs': 200,
                   'batch-size':200,
                   'learning-rate':1e-4}

base_tuning_job_name = 'layer-tuning-' + hyperparameters['net-type']

tags = [{'Key': 'net-type',
         'Value': hyperparameters['net-type']}]

# For GCN
# hyperparameter_ranges = {'gcn-hidden-feats': CategoricalParameter([32,64,128,256,512]),
#                          'classifier-hidden-feats': CategoricalParameter([32,64,128,256,512]),
#                          'num-hidden-layers': CategoricalParameter([2,3,4,5,6])}

hyperparameter_ranges = {'num-hidden-layers': CategoricalParameter([2,3,4,5,6])}

# For GAT
# hyperparameter_ranges = {'gat-hidden-feats': CategoricalParameter([32,64,128]),
#                          'classifier-hidden-feats': CategoricalParameter([64,128,256,512]),
#                          'num-head-layers': CategoricalParameter([2,3,4,5,6]),
#                          'head-output-size': CategoricalParameter([2,3,4,5,6])}


objective_metric_name = 'val:f1_score'
objective_type = 'Maximize'
metric_definitions = [{"Name": "train:loss", 
                       "Regex": "train_loss:(\d+.\d+)"}, 
                      {"Name": "test:auc_roc", 
                       "Regex": "test_score:(\d+.\d+)"},
                      {"Name": "val:auc_roc", 
                       "Regex": "auc_roc:(\d+.\d+)"},
                      {"Name": "val:f1_score", 
                       "Regex": "f1_score:(\d+.\d+)"}
                     ]

estimator = Estimator(role=role,
                      train_instance_count=1,
                       train_instance_type=instance_type,
                      image_name=ecr_image,
                      hyperparameters=hyperparameters,
                     output_path = output_path,
                     train_max_run=max_run_time,
                     metric_definitions=metric_definitions,
                     tags=tags)

tuner = HyperparameterTuner(estimator,
                            objective_metric_name,
                            hyperparameter_ranges,
                            metric_definitions,
                            max_jobs=max_jobs,
                            base_tuning_job_name = base_tuning_job_name,
                            max_parallel_jobs=max_parallel_jobs,
                            objective_type=objective_type)

tuner.fit(data_location)